# Example Notebook to load multiple dataset from different STAC catalogs

### import packages

In [ ]:
# Created by Etiënne Kras, 24-10-2024, using geo_env

import os
import pandas as pd
import pystac_client
import xarray as xr
import rioxarray as rio
import numpy as np

from copy import deepcopy
from typing import List, Dict

### Functions

In [ ]:
# define function(s)

# function to put items in dataframe
def items_to_dataframe(items: List[Dict]) -> pd.DataFrame:
    """STAC items to Pandas dataframe.

    Args:
        items (List[Dict]): _description_

    Returns:
        pd.DataFrame: _description_
    """
    _items = []
    for i in items:
        _i = deepcopy(i)
        # _i['geometry'] = shape(_i['geometry'])
        # ...  # for example, drop some attributes that you're not interested in
        _items.append(_i)
    df = pd.DataFrame(pd.json_normalize(_items))
    # for field in ["properties.datetime"]:
    #     if field in df:
    #         df[field] = pd.to_datetime(df[field])
    # df = df.sort_values("properties.datetime")
    return df

### Open data from first catalog (future shorelines & SLR projections AR6)

In [ ]:
# load the CoCliCo STAC catalog
catalog = pystac_client.Client.open(
    "https://storage.googleapis.com/coclico-data-public/coclico/coclico-stac/catalog.json"
)
# catalog
# see visually here: radiantearth.github.io/stac-browser/#/external/storage.googleapis.com/coclico-data-public/coclico/coclico-stac-4oct/catalog.json

# list the datasets present in the catalog, we are interested in the slp5 and slp6 sets
list(catalog.get_children())

In [ ]:
ar6_col = catalog.get_child("slp6")
parquet_asset = ar6_col.get_assets()["geoparquet-stac-items"]

# Load the parquet file into memory
ar6_parquet_df = pd.read_parquet(parquet_asset.href)
ar6_parquet_df.head()

In [ ]:
%%time
# read STAC items as Pandas dataframe

# TODO: how to speed up?? 
# select one ens ..

# AR6, takes a while +/- 6 min
ar6_col = catalog.get_child("slp6")
items_ar6 = list(ar6_col.get_items()) # this is slow as we need to list all items
items_ar6_df = items_to_dataframe([i.to_dict() for i in items_ar6])

In [ ]:
yr = 2100  # set year
ens = 50  # set ensemble [0-100]
var = "slr"  # set variable
ccs6 = "1-26"  # set climate change scenario for AR6

In [ ]:
# filter items in datasets

# define variables
ens_list = ["5", "50", "95"]  # ensemble list to look into
yrs_list = np.arange(1970, 2200, 10)  # years to look into (step of 10 years from 1970)
key_list = ["CCS", "YRS", "ENS"]

# index AR6 dataframe on criteria
fil_idx6 = []
AR6_dict = {key: [] for key in key_list}
for idx, i in enumerate(items_ar6_df.id):
    enss = str(i).split("/")[1].split("ens")[-1]  # ensemble
    yrs = int(str(i).split("/")[2][0:4])  # yrs
    ccs = str(i).split("/")[0].split("=")[-1] # ccs
    if enss in [str(float(x)) for x in ens_list] and yrs in yrs_list:  # constraining read ensembles and years
        AR6_dict["CCS"].append(ccs)
        AR6_dict["YRS"].append(yrs)
        AR6_dict["ENS"].append(enss)
        fil_idx6.append(idx)

# filter AR6 dataframe and STAC items on index
items_df_fil6 = items_ar6_df.filter(items = fil_idx6, axis=0)
items_fil6 = [items_ar6[i] for i in fil_idx6]

In [ ]:
# get data

# get AR6 collection and item href
for i in items_fil6:
    if i.id == r"ssp=%s/%s_ens%s/%s.tif" % (ccs6, var, float(ens), yr):
        ar6_item_href = i.assets["data"].href

ar6_item = rio.open_rasterio(ar6_item_href, masked=True)
ar6_item_corr = ar6_item / 1000
ar6_item_corr.plot()

# cbar limits
# vmin = max(
#     min(np.nanmin(ar5_item), np.nanmin(ar6_item_corr)), -0.2
# )  # bound to -0.2 if smaller than this value
# vmax = max(np.nanmax(ar5_item), np.nanmax(ar6_item_corr))

# colormap
#cwd = pathlib.Path().resolve().parent
#slev_divl = np.loadtxt(str(pathlib.Path.joinpath(cwd, r"src/coclico/colormaps/slev_div.txt")))
#slev_div = mcolors.LinearSegmentedColormap.from_list('slev_div', slev_divl/255)

In [ ]:
sc_col = catalog.get_child("sc")
href_sc = sc_col.assets["data"].href

# Printing the dataset object shows the storm surge level consists of three dimensions.
# Every storm surge level is associated with a certain station, scenario and revisting period.
ds = xr.open_zarr(href_sc)
ds

In [ ]:
# TODO: plot on map

### Open data from second catalog (world pop & subsidence)

In [ ]:
# load the GCA SOTC STAC catalog
catalog2 = pystac_client.Client.open(
    "https://storage.googleapis.com/dgds-data-public/gca/SOTC/gca-sotc/catalog.json"
)
# catalog
# see visually here: https://radiantearth.github.io/stac-browser/#/external/storage.googleapis.com/dgds-data-public/gca/SOTC/gca-sotc/catalog.json

# list the datasets present in the catalog, we are interested in the slp5 and slp6 sets
list(catalog2.get_children())

In [ ]:
%%time
# read STAC items as Pandas dataframe

# TODO: how to speed up?? 
# select one ens ..

# AR6, takes a while +/- 6 min
sub_col = catalog2.get_child("Haz-Land_Sub_2040_COGs")
items_sub = list(sub_col.get_items()) # this is slow as we need to list all items
items_sub_df = items_to_dataframe([i.to_dict() for i in items_sub])

In [ ]:
# TODO: plot on map

In [ ]:
# parquet file

pop_col = catalog2.get_child("Exp_world_pop_parquet")

# TODO: open parquet file
href_pop = pop_col.assets["data"].href

In [ ]:
pd.read_parquet('example_pa.parquet', engine='pyarrow')

In [ ]:
# TODO: plot on map